##PROJEKAT VELIKI PODACI 
##VUK STANISIC RN46/2019

In [0]:
import pyspark
from pyspark.sql.functions import regexp_replace, col, lower, concat_ws, substring, trim, split, when, round, abs as p_abs, desc
from pyspark.sql.functions import year, month, dayofmonth, to_timestamp, date_format
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

#Kolone u tabeli i njihovo znacenje
##Izvor (source): https://www.kaggle.com/sanjeetsinghnaik/premier-league-matches-20142020
* **home** - (String) : Home team name (Domaci)
* **away** - (String) : Away team name  (Gosti)
* **score** - (String) : Full time score (Konacan rezultat)
* **excitement** - (float) : Match exitement rating out of 10
* **ht_rating** - (float) : Home team rating out of 10
* **at_rating** - (float) : Home team rating out of 10
* **ht_possession** - (int) : Home team possession %  (Posed lopte domaci)
* **at_possession** - (int) : Away team possession %  (Posed lopte gosti)
* **ht_pass** - (float) : Home team pass success %
* **at_pass** - (float) : Away team pass success %
* **ht_fouls** - (float) : Home team number of fouls
* **at_fouls** - (float) : Away team number of fouls
* **ht_shots** - (float) : Home team number of shots
* **at_shots** - (float) : Away team number of shots

#### PODACI I KOLONE u tabeli
Menjanje imena kolonama, odbavivanje nepotrebnih kolona ,,DFa" koji nam nece biti potrebni prilikom analize

In [0]:
# File location and type
file_location = "/FileStore/tables/prem2015.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)


temp_table_name = "prem2015_csv"

df.createOrReplaceTempView(temp_table_name)



# File location and type
file_location = "/FileStore/tables/prem2020.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)


temp_table_name = "prem2020_csv"

df.createOrReplaceTempView(temp_table_name)

Home Team,Away Team,Score,Half Time Score,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards
MAN UTD,TOTTENHAM,1-0,1-0,3.1,6.8,6.4,50,50,4.0,1.0,9.0,4.0,1.0,25.0,82.0,8.0,30.0,12.0,2.0,0.0,0.0,2.0,4.0,9.0,3.0,2.0,24.0,80.0,11.0,20.0,12.0,3.0,0.0,0.0
EVERTON,WATFORD,2-2,0-1,7.4,6.1,6.9,67,33,3.0,5.0,10.0,2.0,8.0,30.0,81.0,20.0,25.0,7.0,1.0,0.0,0.0,3.0,5.0,11.0,3.0,2.0,14.0,70.0,29.0,53.0,13.0,2.0,0.0,0.0
NORWICH,CRYSTAL PALACE,1-3,0-1,6.6,5.2,7.4,63,37,9.0,6.0,17.0,2.0,1.0,23.0,83.0,10.0,26.0,14.0,1.0,0.0,0.0,2.0,7.0,11.0,2.0,4.0,19.0,74.0,8.0,34.0,20.0,0.0,0.0,0.0
LEICESTER CITY,SUNDERLAND,4-2,3-0,7.8,6.6,4.9,44,56,8.0,8.0,20.0,4.0,6.0,21.0,73.0,16.0,21.0,13.0,2.0,0.0,0.0,4.0,5.0,11.0,2.0,3.0,25.0,79.0,25.0,14.0,17.0,4.0,0.0,0.0
BOURNEMOUTH,ASTON VILLA,0-1,0-0,3.2,5.8,7.1,58,42,7.0,2.0,11.0,2.0,6.0,30.0,85.0,17.0,16.0,13.0,3.0,0.0,0.0,2.0,3.0,7.0,2.0,3.0,19.0,79.0,17.0,39.0,13.0,4.0,0.0,0.0
CHELSEA,SWANSEA,2-2,2-1,8.5,5.2,7.2,52,48,5.0,3.0,11.0,3.0,4.0,24.0,80.0,14.0,14.0,15.0,1.0,0.0,1.0,2.0,10.0,17.0,5.0,8.0,22.0,81.0,21.0,29.0,16.0,3.0,0.0,0.0
NEWCASTLE,SOUTHAMPTON,2-2,1-1,6.8,6.4,6.4,56,44,3.0,4.0,9.0,2.0,6.0,34.0,82.0,14.0,24.0,9.0,2.0,0.0,0.0,7.0,4.0,14.0,3.0,6.0,19.0,79.0,21.0,32.0,12.0,4.0,0.0,0.0
ARSENAL,WEST HAM,0-2,0-1,4.7,6.0,7.1,62,38,7.0,6.0,22.0,9.0,5.0,18.0,87.0,32.0,18.0,12.0,1.0,0.0,0.0,2.0,4.0,8.0,2.0,4.0,21.0,71.0,19.0,25.0,9.0,3.0,0.0,0.0
STOKE,LIVERPOOL,0-1,0-0,3.1,6.0,6.8,47,53,4.0,1.0,7.0,2.0,3.0,25.0,73.0,16.0,16.0,9.0,2.0,0.0,0.0,1.0,3.0,8.0,4.0,5.0,22.0,79.0,21.0,33.0,16.0,4.0,0.0,0.0
WEST BROM,MAN CITY,0-3,0-2,5.4,5.3,8.2,31,69,6.0,2.0,9.0,1.0,6.0,16.0,81.0,6.0,21.0,12.0,4.0,0.0,0.0,2.0,7.0,20.0,11.0,6.0,12.0,91.0,12.0,21.0,9.0,1.0,0.0,0.0


Home Team,Away Team,Score,Half Time Score,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards
FULHAM,ARSENAL,0-3,0-1,5.2,5.2,8.1,45,55,2.0,2.0,5.0,1.0,2.0,20.0,88.0,9.0,7.0,12.0,2.0,0.0,0.0,4.0,6.0,13.0,3.0,3.0,21.0,90.0,6.0,10.0,12.0,2.0,0.0,0.0
CRYSTAL PALACE,SOUTHAMPTON,1-0,1-0,3.0,5.8,6.5,30,70,2.0,3.0,5.0,1.0,7.0,33.0,51.0,27.0,22.0,14.0,2.0,0.0,0.0,3.0,5.0,9.0,1.0,3.0,32.0,79.0,30.0,13.0,11.0,1.0,0.0,0.0
LIVERPOOL,LEEDS UTD,4-3,3-2,9.1,6.8,5.8,48,52,3.0,6.0,22.0,13.0,9.0,34.0,76.0,15.0,11.0,9.0,1.0,0.0,0.0,1.0,3.0,6.0,2.0,0.0,25.0,76.0,10.0,34.0,6.0,0.0,0.0,0.0
WEST HAM,NEWCASTLE,0-2,0-0,5.3,5.6,6.7,58,42,5.0,3.0,15.0,7.0,8.0,17.0,83.0,16.0,16.0,13.0,2.0,0.0,0.0,7.0,2.0,15.0,6.0,7.0,10.0,78.0,22.0,27.0,7.0,2.0,0.0,0.0
WEST BROM,LEICESTER CITY,0-3,0-0,4.9,4.5,8.6,37,63,3.0,1.0,7.0,3.0,2.0,22.0,76.0,14.0,12.0,12.0,1.0,0.0,0.0,3.0,7.0,13.0,3.0,5.0,31.0,84.0,9.0,14.0,9.0,1.0,0.0,0.0
TOTTENHAM,EVERTON,0-1,0-0,3.8,6.0,7.2,51,49,1.0,5.0,9.0,3.0,5.0,34.0,85.0,18.0,17.0,15.0,1.0,0.0,0.0,7.0,4.0,15.0,4.0,3.0,17.0,85.0,15.0,35.0,7.0,0.0,0.0,0.0
SHEFFIELD UTD,WOLVES,0-2,0-2,4.7,5.6,6.9,55,45,5.0,2.0,9.0,2.0,12.0,23.0,81.0,31.0,10.0,13.0,2.0,0.0,0.0,5.0,4.0,11.0,2.0,5.0,19.0,78.0,27.0,30.0,7.0,1.0,0.0,0.0
BRIGHTON,CHELSEA,1-3,0-1,5.6,5.3,7.7,52,48,6.0,3.0,13.0,4.0,4.0,24.0,82.0,9.0,8.0,8.0,1.0,0.0,0.0,2.0,5.0,10.0,3.0,3.0,21.0,83.0,13.0,14.0,13.0,0.0,0.0,0.0
BURNLEY,MAN UTD,0-1,0-0,3.3,5.3,7.0,37,63,7.0,0.0,12.0,5.0,3.0,22.0,66.0,12.0,18.0,15.0,1.0,0.0,0.0,2.0,7.0,13.0,4.0,4.0,14.0,84.0,21.0,19.0,13.0,2.0,0.0,0.0
MAN CITY,ASTON VILLA,2-0,0-0,6.0,7.7,5.0,69,31,7.0,9.0,28.0,12.0,16.0,14.0,88.0,21.0,9.0,10.0,0.0,0.0,0.0,4.0,4.0,11.0,3.0,2.0,13.0,71.0,11.0,28.0,9.0,2.0,0.0,0.0


In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `prem2015_csv`



Home Team,Away Team,Score,Half Time Score,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards
MAN UTD,TOTTENHAM,1-0,1-0,3.1,6.8,6.4,50,50,4.0,1.0,9.0,4.0,1.0,25.0,82.0,8.0,30.0,12.0,2.0,0.0,0.0,2.0,4.0,9.0,3.0,2.0,24.0,80.0,11.0,20.0,12.0,3.0,0.0,0.0
EVERTON,WATFORD,2-2,0-1,7.4,6.1,6.9,67,33,3.0,5.0,10.0,2.0,8.0,30.0,81.0,20.0,25.0,7.0,1.0,0.0,0.0,3.0,5.0,11.0,3.0,2.0,14.0,70.0,29.0,53.0,13.0,2.0,0.0,0.0
NORWICH,CRYSTAL PALACE,1-3,0-1,6.6,5.2,7.4,63,37,9.0,6.0,17.0,2.0,1.0,23.0,83.0,10.0,26.0,14.0,1.0,0.0,0.0,2.0,7.0,11.0,2.0,4.0,19.0,74.0,8.0,34.0,20.0,0.0,0.0,0.0
LEICESTER CITY,SUNDERLAND,4-2,3-0,7.8,6.6,4.9,44,56,8.0,8.0,20.0,4.0,6.0,21.0,73.0,16.0,21.0,13.0,2.0,0.0,0.0,4.0,5.0,11.0,2.0,3.0,25.0,79.0,25.0,14.0,17.0,4.0,0.0,0.0
BOURNEMOUTH,ASTON VILLA,0-1,0-0,3.2,5.8,7.1,58,42,7.0,2.0,11.0,2.0,6.0,30.0,85.0,17.0,16.0,13.0,3.0,0.0,0.0,2.0,3.0,7.0,2.0,3.0,19.0,79.0,17.0,39.0,13.0,4.0,0.0,0.0
CHELSEA,SWANSEA,2-2,2-1,8.5,5.2,7.2,52,48,5.0,3.0,11.0,3.0,4.0,24.0,80.0,14.0,14.0,15.0,1.0,0.0,1.0,2.0,10.0,17.0,5.0,8.0,22.0,81.0,21.0,29.0,16.0,3.0,0.0,0.0
NEWCASTLE,SOUTHAMPTON,2-2,1-1,6.8,6.4,6.4,56,44,3.0,4.0,9.0,2.0,6.0,34.0,82.0,14.0,24.0,9.0,2.0,0.0,0.0,7.0,4.0,14.0,3.0,6.0,19.0,79.0,21.0,32.0,12.0,4.0,0.0,0.0
ARSENAL,WEST HAM,0-2,0-1,4.7,6.0,7.1,62,38,7.0,6.0,22.0,9.0,5.0,18.0,87.0,32.0,18.0,12.0,1.0,0.0,0.0,2.0,4.0,8.0,2.0,4.0,21.0,71.0,19.0,25.0,9.0,3.0,0.0,0.0
STOKE,LIVERPOOL,0-1,0-0,3.1,6.0,6.8,47,53,4.0,1.0,7.0,2.0,3.0,25.0,73.0,16.0,16.0,9.0,2.0,0.0,0.0,1.0,3.0,8.0,4.0,5.0,22.0,79.0,21.0,33.0,16.0,4.0,0.0,0.0
WEST BROM,MAN CITY,0-3,0-2,5.4,5.3,8.2,31,69,6.0,2.0,9.0,1.0,6.0,16.0,81.0,6.0,21.0,12.0,4.0,0.0,0.0,2.0,7.0,20.0,11.0,6.0,12.0,91.0,12.0,21.0,9.0,1.0,0.0,0.0


In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `prem2020_csv`

Home Team,Away Team,Score,Half Time Score,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,Home Team On Target Shots,Home Team Total Shots,Home Team Blocked Shots,Home Team Corners,Home Team Throw Ins,Home Team Pass Success %,Home Team Aerials Won,Home Team Clearances,Home Team Fouls,Home Team Yellow Cards,Home Team Second Yellow Cards,Home Team Red Cards,Away Team Off Target Shots,Away Team On Target Shots,Away Team Total Shots,Away Team Blocked Shots,Away Team Corners,Away Team Throw Ins,Away Team Pass Success %,Away Team Aerials Won,Away Team Clearances,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards
FULHAM,ARSENAL,0-3,0-1,5.2,5.2,8.1,45,55,2.0,2.0,5.0,1.0,2.0,20.0,88.0,9.0,7.0,12.0,2.0,0.0,0.0,4.0,6.0,13.0,3.0,3.0,21.0,90.0,6.0,10.0,12.0,2.0,0.0,0.0
CRYSTAL PALACE,SOUTHAMPTON,1-0,1-0,3.0,5.8,6.5,30,70,2.0,3.0,5.0,1.0,7.0,33.0,51.0,27.0,22.0,14.0,2.0,0.0,0.0,3.0,5.0,9.0,1.0,3.0,32.0,79.0,30.0,13.0,11.0,1.0,0.0,0.0
LIVERPOOL,LEEDS UTD,4-3,3-2,9.1,6.8,5.8,48,52,3.0,6.0,22.0,13.0,9.0,34.0,76.0,15.0,11.0,9.0,1.0,0.0,0.0,1.0,3.0,6.0,2.0,0.0,25.0,76.0,10.0,34.0,6.0,0.0,0.0,0.0
WEST HAM,NEWCASTLE,0-2,0-0,5.3,5.6,6.7,58,42,5.0,3.0,15.0,7.0,8.0,17.0,83.0,16.0,16.0,13.0,2.0,0.0,0.0,7.0,2.0,15.0,6.0,7.0,10.0,78.0,22.0,27.0,7.0,2.0,0.0,0.0
WEST BROM,LEICESTER CITY,0-3,0-0,4.9,4.5,8.6,37,63,3.0,1.0,7.0,3.0,2.0,22.0,76.0,14.0,12.0,12.0,1.0,0.0,0.0,3.0,7.0,13.0,3.0,5.0,31.0,84.0,9.0,14.0,9.0,1.0,0.0,0.0
TOTTENHAM,EVERTON,0-1,0-0,3.8,6.0,7.2,51,49,1.0,5.0,9.0,3.0,5.0,34.0,85.0,18.0,17.0,15.0,1.0,0.0,0.0,7.0,4.0,15.0,4.0,3.0,17.0,85.0,15.0,35.0,7.0,0.0,0.0,0.0
SHEFFIELD UTD,WOLVES,0-2,0-2,4.7,5.6,6.9,55,45,5.0,2.0,9.0,2.0,12.0,23.0,81.0,31.0,10.0,13.0,2.0,0.0,0.0,5.0,4.0,11.0,2.0,5.0,19.0,78.0,27.0,30.0,7.0,1.0,0.0,0.0
BRIGHTON,CHELSEA,1-3,0-1,5.6,5.3,7.7,52,48,6.0,3.0,13.0,4.0,4.0,24.0,82.0,9.0,8.0,8.0,1.0,0.0,0.0,2.0,5.0,10.0,3.0,3.0,21.0,83.0,13.0,14.0,13.0,0.0,0.0,0.0
BURNLEY,MAN UTD,0-1,0-0,3.3,5.3,7.0,37,63,7.0,0.0,12.0,5.0,3.0,22.0,66.0,12.0,18.0,15.0,1.0,0.0,0.0,2.0,7.0,13.0,4.0,4.0,14.0,84.0,21.0,19.0,13.0,2.0,0.0,0.0
MAN CITY,ASTON VILLA,2-0,0-0,6.0,7.7,5.0,69,31,7.0,9.0,28.0,12.0,16.0,14.0,88.0,21.0,9.0,10.0,0.0,0.0,0.0,4.0,4.0,11.0,3.0,2.0,13.0,71.0,11.0,28.0,9.0,2.0,0.0,0.0


In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "prem2015_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "prem2020_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df15 = spark.sql("select `Home Team`, `Away Team`, `Score`, `Match Excitement`, `Home Team Rating`, `Away Team Rating`, `Home Team Possession %`, `Away Team Possession %`, `Home Team Pass Success %`, `Away Team Pass Success %`, `Home Team Fouls`, `Away Team Fouls`, `Home Team Total Shots`, `Away Team Total Shots` from prem2015_csv")
df20 = spark.sql("select `Home Team`, `Away Team`, `Score`,  `Match Excitement`, `Home Team Rating`, `Away Team Rating`, `Home Team Possession %`, `Away Team Possession %`, `Home Team Pass Success %`, `Away Team Pass Success %`, `Home Team Fouls`, `Away Team Fouls`, `Home Team Total Shots`, `Away Team Total Shots` from prem2020_csv")

columns = ["home","away","score", "excitement","ht_rating","at_rating","ht_possession","at_possession","ht_pass","at_pass","ht_fouls","at_fouls","ht_shots","at_shots"]

cols1 = df15.columns
cols2 = df20.columns
i = 0
for c1, c2 in zip(cols1, cols2):
    df15 = df15.withColumnRenamed(c1, columns[i])
    df20 = df20.withColumnRenamed(c2, columns[i])
    i = i + 1
    
df15.printSchema()
df20.printSchema()
df15.show(5)
df20.show(5)

df15.cache()
df20.cache()
                

root
 |-- home: string (nullable = true)
 |-- away: string (nullable = true)
 |-- score: string (nullable = true)
 |-- excitement: double (nullable = true)
 |-- ht_rating: double (nullable = true)
 |-- at_rating: double (nullable = true)
 |-- ht_possession: integer (nullable = true)
 |-- at_possession: integer (nullable = true)
 |-- ht_pass: double (nullable = true)
 |-- at_pass: double (nullable = true)
 |-- ht_fouls: double (nullable = true)
 |-- at_fouls: double (nullable = true)
 |-- ht_shots: double (nullable = true)
 |-- at_shots: double (nullable = true)

root
 |-- home: string (nullable = true)
 |-- away: string (nullable = true)
 |-- score: string (nullable = true)
 |-- excitement: double (nullable = true)
 |-- ht_rating: double (nullable = true)
 |-- at_rating: double (nullable = true)
 |-- ht_possession: integer (nullable = true)
 |-- at_possession: integer (nullable = true)
 |-- ht_pass: double (nullable = true)
 |-- at_pass: double (nullable = true)
 |-- ht_fouls: double (

#### Castovanje podataka u odgovarajuci tip
Faulovi i sutevi moraju biti int jer su celobrojne vrednosti

In [0]:
def cast_columns(df):
    df= df.withColumn("ht_shots", col("at_shots").cast("int"))
    df= df.withColumn("at_shots", col("at_shots").cast("int")) 
    df= df.withColumn("ht_fouls", col("ht_fouls").cast("int"))
    df= df.withColumn("at_fouls", col("at_fouls").cast("int"))
    return df

df15 = cast_columns(df15)
df20 = cast_columns(df20)

df20= df20.withColumn("at_possession", col("at_possession").cast("int"))

###SCORE pretvaram u golove
Razdvojicemo na domace i goste i castujemo u int

In [0]:
df15 = df15.withColumn("ht_goals",split(col("score"),"-").getItem(0).cast("int")).withColumn("at_goals",split(col("score"),"-").getItem(1).cast("int")).drop("score")
df20 = df20.withColumn("ht_goals",split(col("score"),"-").getItem(0).cast("int")).withColumn("at_goals",split(col("score"),"-").getItem(1).cast("int")).drop("score")

df15.show(5)
df20.show(5)

+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+
|          home|          away|excitement|ht_rating|at_rating|ht_possession|at_possession|ht_pass|at_pass|ht_fouls|at_fouls|ht_shots|at_shots|ht_goals|at_goals|
+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+
|       MAN UTD|     TOTTENHAM|       3.1|      6.8|      6.4|           50|           50|   82.0|   80.0|      12|      12|       9|       9|       1|       0|
|       EVERTON|       WATFORD|       7.4|      6.1|      6.9|           67|           33|   81.0|   70.0|       7|      13|      11|      11|       2|       2|
|       NORWICH|CRYSTAL PALACE|       6.6|      5.2|      7.4|           63|           37|   83.0|   74.0|      14|      20|      11|      11|       1|       3|
|LEICESTER CITY|    SUNDERLAND|   

####ISHOD
1 ili x ili 2 u odnosu na to koji tim je na utakmici postigao koliko golova

In [0]:
df15 = df15.withColumn("result", when((col("ht_goals") > col("at_goals")), "1").otherwise(when((col("ht_goals") < col("at_goals")), "2").otherwise("x")))
df20 = df20.withColumn("result", when((col("ht_goals") > col("at_goals")), "1").otherwise(when((col("ht_goals") < col("at_goals")), "2").otherwise("x")))

df15.show(5)
df20.show(5)

+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+------+
|          home|          away|excitement|ht_rating|at_rating|ht_possession|at_possession|ht_pass|at_pass|ht_fouls|at_fouls|ht_shots|at_shots|ht_goals|at_goals|result|
+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+------+
|       MAN UTD|     TOTTENHAM|       3.1|      6.8|      6.4|           50|           50|   82.0|   80.0|      12|      12|       9|       9|       1|       0|     1|
|       EVERTON|       WATFORD|       7.4|      6.1|      6.9|           67|           33|   81.0|   70.0|       7|      13|      11|      11|       2|       2|     x|
|       NORWICH|CRYSTAL PALACE|       6.6|      5.2|      7.4|           63|           37|   83.0|   74.0|      14|      20|      11|      11|       1|       3|

####Shot Accuracy
Odnos suteva i golova po timu na svakoj utakmici

In [0]:
df15 = df15.withColumn("ht_accuracy", round(col("ht_goals") / col("ht_shots"), 2))
df15 = df15.withColumn("at_accuracy", round(col("at_goals") / col("at_shots"), 2))
df20 = df15.withColumn("ht_accuracy", round(col("ht_goals") / col("ht_shots"), 2))
df20 = df20.withColumn("at_accuracy", round(col("at_goals") / col("at_shots"), 2))

df15.show(5)
df20.show(5)


+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+------+-----------+-----------+
|          home|          away|excitement|ht_rating|at_rating|ht_possession|at_possession|ht_pass|at_pass|ht_fouls|at_fouls|ht_shots|at_shots|ht_goals|at_goals|result|ht_accuracy|at_accuracy|
+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+------+-----------+-----------+
|       MAN UTD|     TOTTENHAM|       3.1|      6.8|      6.4|           50|           50|   82.0|   80.0|      12|      12|       9|       9|       1|       0|     1|       0.11|        0.0|
|       EVERTON|       WATFORD|       7.4|      6.1|      6.9|           67|           33|   81.0|   70.0|       7|      13|      11|      11|       2|       2|     x|       0.18|       0.18|
|       NORWICH|CRYSTAL PALACE|       6.

####Gol razlika
Gol razlika na utakmici. Koristicemo je kasnije za korelaciju tim - pobeda/ poraz- gol_razlika

In [0]:
df15 = df15.withColumn("goal_difference", p_abs(col("ht_goals") - col("at_goals")))
df20 = df20.withColumn("goal_difference", p_abs(col("ht_goals") - col("at_goals")))

df15.show(5)
df20.show(5)

+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+------+-----------+-----------+---------------+
|          home|          away|excitement|ht_rating|at_rating|ht_possession|at_possession|ht_pass|at_pass|ht_fouls|at_fouls|ht_shots|at_shots|ht_goals|at_goals|result|ht_accuracy|at_accuracy|goal_difference|
+--------------+--------------+----------+---------+---------+-------------+-------------+-------+-------+--------+--------+--------+--------+--------+--------+------+-----------+-----------+---------------+
|       MAN UTD|     TOTTENHAM|       3.1|      6.8|      6.4|           50|           50|   82.0|   80.0|      12|      12|       9|       9|       1|       0|     1|       0.11|        0.0|              1|
|       EVERTON|       WATFORD|       7.4|      6.1|      6.9|           67|           33|   81.0|   70.0|       7|      13|      11|      11|       2|       2|     x| 

In [0]:
# export

df15.write.csv("df15_csv", header= True, mode="overwrite")
df20.write.csv("df20_csv", header= True, mode="overwrite")